In [ ]:
import pandas as pd
from datetime import timedelta
import psycopg2
from psycopg2 import OperationalError

def connect_to_db():
    try:
        connection = psycopg2.connect(
            user="user",
            password="example",
            host="127.0.0.1",
            port="5432",
            database="mydatabase"
        )
        #print("Connection to PostgreSQL DB successful")
        return connection
    except OperationalError as e:
        print(f"The error '{e}' occurred")
        return None


def query_spot_data(day):
    query = f'''
        SELECT symbol, bid1Price, lastprice, time FROM spot_ticker_data
        where symbol like '%USDC' and time >= TIMESTAMP '{day}' 
        ORDER BY time desc;'''
    with connect_to_db() as conn:
        return pd.read_sql_query(query, conn)

def query_option_data(day, day_new_str, active, type_of_option):
    query = f'''
        SELECT symbol, markprice, underlyingprice, lastprice, time FROM option_ticker_data
        where symbol like '{active}-{day_new_str}%{type_of_option}' and time <= TIMESTAMP '{day+timedelta(days=1)}' and time >= TIMESTAMP '{day}' and markprice > 0.01
        ORDER BY time desc;'''
    with connect_to_db() as conn:
        return pd.read_sql_query(query, conn)

In [ ]:
#symbol like 'BTC%{'C'}' and 
query = f'''SELECT max(start_id) FROM option_ticker_data  '''
with connect_to_db() as conn:
    ans = pd.read_sql_query(query, conn)